<a href="https://colab.research.google.com/github/eduardoseity/Loterias/blob/main/Notebooks/Loterias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Classes

In [ ]:
from abc import ABC, abstractmethod
import results
import random
import pandas as pd
import numpy as np

class ILotto(ABC):
    @abstractmethod
    def min_bet(self):
        pass
    @abstractmethod
    def max_bet(self):
        pass
    @abstractmethod
    def prices(self):
        pass
    @abstractmethod
    def winning_categories(self):
        pass
    @abstractmethod
    def prizes(self):
        pass
    @abstractmethod
    def lower_upper_number(self):
        pass
    @abstractmethod
    def modalidade(self):
        pass
    @abstractmethod
    def drawn_number_columns(self):
        pass
    @abstractmethod
    def results_dataframe(self):
        pass
    @abstractmethod
    def drawn_numbers_qty(self):
        pass
    @abstractmethod
    def numbers_position(self):
        pass

    @abstractmethod
    def validate_bet(self, bet:list) -> bool:
        if len(bet) < self.min_bet or len(bet) > self.max_bet: # Test the bet numbers quantity
            raise ValueError(f'Quantidade de números apostados deve estar entre {self.min_bet} e {self.max_bet}')
        elif len(set(bet)) != len(bet): # Check if values are unique
            raise ValueError('Existem números repetidos na aposta')
        elif sorted(bet)[0] < self.lower_upper_number[0] or sorted(bet)[-1] > self.lower_upper_number[1]: # Check if values are in defined limits
            raise ValueError('Existem números fora do intervalo permitido')
        else:
            return True

    def check_result(self, result:np.ndarray, bet_numbers:list) -> list:
        drawn_numbers = result[self.drawn_number_columns]
        winner_numbers = [x for x in bet_numbers if x in drawn_numbers]
        return winner_numbers

    def check_results(self, results_array:np.ndarray, bet_numbers:list) -> list:
        if not isinstance(bet_numbers[0], list): raise TypeError('O parâmetro bet_numbers deve ser uma lista de listas')
        winner_numbers = []
        for r in results_array:
            for b in bet_numbers:
                winner_numbers.append(self.check_result(r, b))
        return winner_numbers

    @abstractmethod
    def update_results(self, overwrite:bool = False) -> pd.DataFrame:
        results.get_results(self.modalidade, overwrite)
        self.results_dataframe = pd.read_csv(f'assets/results/{self.modalidade}.csv')
        return self.results_dataframe

    def generate_random(self, num:int) -> list:
        if num < self.min_bet or num > self.max_bet: raise ValueError(f"O parâmetro 'num' deve ser um número entre {self.min_bet} e {self.max_bet}")
        remaining_numbers = list(range(self.lower_upper_number[0], self.lower_upper_number[1]+1))
        bet = []
        for x in range(num):
            index = random.randint(0, len(remaining_numbers)-1)
            bet.append(remaining_numbers[index])
            remaining_numbers.remove(bet[-1])

        return bet

    def get_last_result(self) -> np.ndarray:
        self.update_results()
        return self.results_dataframe[-1:].to_numpy()

    def get_results(self, interval:list=None) -> np.ndarray:
        self.update_results()
        if interval != None:
            return self.results_dataframe.iloc[interval].to_numpy()
        else:
            return self.results_dataframe.to_numpy()

class IRule(ABC):
    @abstractmethod
    def params(self):
        pass

    @abstractmethod
    def check(self):
        pass

class RuleEvenOddPercentage(IRule):
    @property
    def params(self):
        return self.__params
    @params.setter
    def params(self, value):
        self.__params = value

    def check(self, params):
        self.params = params
        result = []
        total = len(params[0])
        for x in self.params:
            even = [n for n in x if n % 2 == 0]
            odd = [n for n in x if n % 2 != 0]
            result.append((len(even), len(odd), len(even)/total, len(odd)/total))
        return result

class Wallet:
    def __init__(self, name:str, negative_allowed:bool = False) -> None:
        self.__balance = 0
        self.name = name
        self.__negative_allowed = negative_allowed

    def deposit(self, amount:float) -> float:
        if not isinstance(amount, float) and not isinstance(amount, int): raise TypeError
        self.__balance += amount
        return self.__balance

    def withdraw(self, amount:float) -> float:
        if not isinstance(amount, float) and not isinstance(amount, int): raise TypeError
        if amount <= 0: raise ValueError

        new_balance = self.__balance - amount
        if not self.__negative_allowed and new_balance < 0:
            raise Exception('Saldo insuficiente!')

        self.__balance = new_balance
        return self.__balance

    @property
    def balance(self):
        pass
    @balance.getter
    def balance(self):
        return self.__balance

class Megasena(ILotto):
    def __init__(self) -> None:
        self.name = 'Megasena'
        self.update_results()

    @property
    def min_bet(self):
        return 6
    @property
    def max_bet(self):
        return 20
    @property
    def winning_categories(self):
        return [4,5,6]
    @property
    def prices(self):
        return [5, 35, 140, 420, 1050, 2310, 4620, 8580, 15015, 25025, 40040, 61880, 92820, 135660, 193800]
    @property
    def prizes(self):
        return []
    @property
    def lower_upper_number(self):
        return (1,60)
    @property
    def modalidade(self):
        return 'Mega-Sena'
    @property
    def drawn_number_columns(self):
        return [2,3,4,5,6,7]
    @property
    def results_dataframe(self):
        return self._results_dataframe
    @results_dataframe.setter
    def results_dataframe(self, value:pd.DataFrame()):
        self._results_dataframe = value
    @property
    def drawn_numbers_qty(self):
        return 6
    @property
    def numbers_position(self):
        return [
            list(range(1,11)),
            list(range(11,21)),
            list(range(21,31)),
            list(range(31,41)),
            list(range(41,51)),
            list(range(51,61)),
        ]

    def update_results(self, overwrite: bool = False) -> pd.DataFrame:
        return super().update_results(overwrite)

    def validate_bet(self, bet: list):
        return super().validate_bet(bet)

class Lotofacil(ILotto):

    def __init__(self) -> None:
        self.name = 'Lotofacil'
        self.update_results()

    @property
    def min_bet(self):
        return 15
    @property
    def max_bet(self):
        return 20
    @property
    def winning_categories(self):
        return [11,12,13,14,15]
    @property
    def prices(self):
        return [3, 48, 408, 2448, 11628, 46512]
    @property
    def prizes(self):
        return []
    @property
    def lower_upper_number(self):
        return (1,25)
    @property
    def modalidade(self):
        return 'Lotofácil'
    @property
    def drawn_number_columns(self):
        return [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
    @property
    def results_dataframe(self):
        return self._results_dataframe
    @results_dataframe.setter
    def results_dataframe(self, value:pd.DataFrame()):
        self._results_dataframe = value
    @property
    def drawn_numbers_qty(self):
        return 15
    @property
    def numbers_position(self):
        return [
            list(range(1,6)),
            list(range(6,11)),
            list(range(11,16)),
            list(range(16,21)),
            list(range(21,26)),
        ]

    def update_results(self, overwrite: bool = False) -> pd.DataFrame:
        return super().update_results(overwrite)

    def validate_bet(self, bet: list):
        return super().validate_bet(bet)

class Quina(ILotto):

    def __init__(self) -> None:
        self.name = 'Quina'
        self.update_results()

    @property
    def min_bet(self):
        return 5
    @property
    def max_bet(self):
        return 15
    @property
    def winning_categories(self):
        return [2,3,4,5]
    @property
    def prices(self):
        return [2.5,15,52.5,140,315,630,1155,1980,3217.5,5005,7507.5]
    @property
    def prizes(self):
        return []
    @property
    def lower_upper_number(self):
        return (1,80)
    @property
    def modalidade(self):
        return 'Quina'
    @property
    def drawn_number_columns(self):
        return [2,3,4,5,6]
    @property
    def results_dataframe(self):
        return self._results_dataframe
    @results_dataframe.setter
    def results_dataframe(self, value:pd.DataFrame()):
        self._results_dataframe = value
    @property
    def drawn_numbers_qty(self):
        return 5
    @property
    def numbers_position(self):
        return [
            list(range(1,11)),
            list(range(11,21)),
            list(range(21,31)),
            list(range(31,41)),
            list(range(41,51)),
            list(range(51,61)),
            list(range(61,71)),
            list(range(71,81))
        ]

    def update_results(self, overwrite: bool = False) -> pd.DataFrame:
        return super().update_results(overwrite)

    def validate_bet(self, bet: list):
        return super().validate_bet(bet)

class Lotomania(ILotto):

    def __init__(self) -> None:
        self.name = 'Lotomania'
        self.update_results()

    @property
    def min_bet(self):
        return 50
    @property
    def max_bet(self):
        return 50
    @property
    def winning_categories(self):
        return [0,15,16,17,18,19,20]
    @property
    def prices(self):
        return [3]
    @property
    def prizes(self):
        return []
    @property
    def lower_upper_number(self):
        return (1,100)
    @property
    def modalidade(self):
        return 'Lotomania'
    @property
    def drawn_number_columns(self):
        return [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21]
    @property
    def results_dataframe(self):
        return self._results_dataframe
    @results_dataframe.setter
    def results_dataframe(self, value:pd.DataFrame()):
        self._results_dataframe = value
    @property
    def drawn_numbers_qty(self):
        return 20
    @property
    def numbers_position(self):
        return [
            list(range(1,11)),
            list(range(11,21)),
            list(range(21,31)),
            list(range(31,41)),
            list(range(41,51)),
            list(range(51,61)),
            list(range(61,71)),
            list(range(71,81)),
            list(range(81,91)),
            list(range(91,101)),
        ]

    def update_results(self, overwrite: bool = False) -> pd.DataFrame:
        df = super().update_results(overwrite)
        # Change zero to 100 (original dataset uses two cases, but it is the same number in the game)
        df.iloc[:,self.drawn_number_columns] = df.iloc[:,self.drawn_number_columns].replace([0],[100])
        return df

    def validate_bet(self, bet: list):
        return super().validate_bet(bet)
